# Creando un Modelo Personalizado para Span Categorizer

* spaCy puede ser actualizado a partir de objetos `Doc` con anotaciones como span categorizer.
* Para colocar las entidades al `Doc` adicionamos un span a `doc.spans["sc"]`.
* El `DocBin` es un contenedor para almacenar y serializar objetos `Doc` de manera eficiente. Puede instanciarlo con una lista de objetos `Doc` y llamar a su método `to_disk` para guardarlo en un archivo binario `.spacy`.

## Generando Datos de Entrenamiento

In [1]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("es_core_news_lg",
                 disable = ["parser", "attribute_ruler", "lemmatizer", "ner"]
                )

matcher = Matcher(nlp.vocab)

un = ["km", "au"]
pattern = [{"POS": "NUM"}, 
           {"OP": "?", "TEXT": {"IN": un}}, 
           {"LOWER": {"IN": ["a", "hasta", "-"]}}, 
           {"POS": "NUM"}, 
           {"TEXT": {"IN": un}}     
           ]

pattern_2 = [{"POS": "NUM"},
             {"TEXT": {"IN": un}}
             ]

planets = ["júpiter", "saturno", "urano", "neptuno"]
pattern_3 = [{"TEXT": {"IN": planets}}]
pattern_3_2 = [{"TEXT": "planeta"},
               {"TEXT": {"IN": ["9", "nueve"]}}
               ]

matcher.add("Intervalo", [pattern])
matcher.add("Magnitud", [pattern_2])
matcher.add("Planeta", [pattern_3, pattern_3_2])

TEXTS = ["El Planeta 9 es un planeta hipotético en el sistema solar externo. ",
         "El semieje mayor propuesto de su órbita varía entre 700 - 1500 au. ",
         'Brown ha declarado: "En realidad lo llamamos “Fattie” (Gordito) cuando estamos hablando entre nosotros". ',
         "La capacidad para detectar el Planeta Nueve dependería de su ubicación y características. ",
         "La existencia de este planeta puede inferirse por el comportamiento de un grupo de objetos transneptunianos.",
         "Probablemente se formó más cerca del Sol, pero fue finalmente empujado más lejos por Júpiter o Saturno "
         "durante la época nebular, arrojándolo a los extremos exteriores del Sistema Solar",
         "El Planeta Nueve es similar en tamaño y composición a los gigantes de hielo Urano y Neptuno",
         "Inicialmente, Trujillo y Shepherd propusieron una órbita circular entre 200 - 300 au.",
         "Las perturbaciones de los cuatro planetas gigantes conocidos del sistema solar "
         "(Júpiter, Saturno, Urano y Neptuno) deberían haber dejado alejados del perihelio de los doce TNOs."
        ]

TEXTS = [i.lower() for i in TEXTS]
    
TRAINING_DATA = []

# Crear un objeto Doc para cada frase en TEXTS
for doc in nlp.pipe(TEXTS):
    # El match es una lista de tripletes en la forma (match id, posición inicial, posición final)
    spans = []
    pattern_names = []
    for match_id, start, end in matcher(doc):
        pattern_names.append(nlp.vocab.strings[match_id])
        # Hacer la correspondencia con el doc
        spans.append(doc[start:end])
    # Obtener las tuplas (caracter inicial, caracter final, rótulo) de las particiones con correspondencia
    entities = []
    for span, pattern_name in zip(spans, pattern_names):
        entities.append((span.start_char, span.end_char, pattern_name))
    # Formatear las correspondencias como una tupla (doc.text, entities)
    training_example = (doc.text, {"entities": entities})
    # Adicionar el ejemplo a los dados de entrenamiento
    TRAINING_DATA.append(training_example)
    
print(TRAINING_DATA)    

[('el planeta 9 es un planeta hipotético en el sistema solar externo. ', {'entities': [(3, 12, 'Planeta')]}), ('el semieje mayor propuesto de su órbita varía entre 700 - 1500 au. ', {'entities': [(52, 65, 'Intervalo'), (58, 65, 'Magnitud')]}), ('brown ha declarado: "en realidad lo llamamos “fattie” (gordito) cuando estamos hablando entre nosotros". ', {'entities': []}), ('la capacidad para detectar el planeta nueve dependería de su ubicación y características. ', {'entities': [(30, 43, 'Planeta')]}), ('la existencia de este planeta puede inferirse por el comportamiento de un grupo de objetos transneptunianos.', {'entities': []}), ('probablemente se formó más cerca del sol, pero fue finalmente empujado más lejos por júpiter o saturno durante la época nebular, arrojándolo a los extremos exteriores del sistema solar', {'entities': [(85, 92, 'Planeta'), (95, 102, 'Planeta')]}), ('el planeta nueve es similar en tamaño y composición a los gigantes de hielo urano y neptuno', {'entities': [(3,

In [2]:
import pickle

with open("training_data_spancat.pickle", "wb") as fp:
    pickle.dump(TRAINING_DATA, fp)

In [3]:
TRAINING_DATA

[('el planeta 9 es un planeta hipotético en el sistema solar externo. ',
  {'entities': [(3, 12, 'Planeta')]}),
 ('el semieje mayor propuesto de su órbita varía entre 700 - 1500 au. ',
  {'entities': [(52, 65, 'Intervalo'), (58, 65, 'Magnitud')]}),
 ('brown ha declarado: "en realidad lo llamamos “fattie” (gordito) cuando estamos hablando entre nosotros". ',
  {'entities': []}),
 ('la capacidad para detectar el planeta nueve dependería de su ubicación y características. ',
  {'entities': [(30, 43, 'Planeta')]}),
 ('la existencia de este planeta puede inferirse por el comportamiento de un grupo de objetos transneptunianos.',
  {'entities': []}),
 ('probablemente se formó más cerca del sol, pero fue finalmente empujado más lejos por júpiter o saturno durante la época nebular, arrojándolo a los extremos exteriores del sistema solar',
  {'entities': [(85, 92, 'Planeta'), (95, 102, 'Planeta')]}),
 ('el planeta nueve es similar en tamaño y composición a los gigantes de hielo urano y neptuno',

## Entrenamiento

In [4]:
import os
os.chdir("model_spancat")

In [5]:
!pwd

/home/jessi/Documentos/Data_Science/spacy/model_spancat


In [6]:
#!python -m spacy init config ./config.cfg --lang es --pipeline spancat -F

In [7]:
# Modified config.cfg and with "train" and "dev" paths defined
!cat ./config.cfg

[paths]
train = train.spacy
dev = dev.spacy
vectors = null
init_tok2vec = null

[system]
gpu_allocator = null
seed = 0

[nlp]
lang = "es"
pipeline = ["tok2vec", "spancat"]
batch_size = 1000
disabled = []
before_creation = null
after_creation = null
after_pipeline_creation = null
tokenizer = {"@tokenizers":"spacy.Tokenizer.v1"}

[components]
[components.tok2vec]
source="es_core_news_lg"

[components.spancat]
factory = "spancat"
spans_key = "sc"

[components.spancat.suggester]
@misc = "spacy.ngram_range_suggester.v1"
min_size = 1
max_size = 10

[components.spancat.model]
@architectures = "spacy.SpanCategorizer.v1"
scorer = {"@layers": "spacy.LinearLogistic.v1"}

[components.spancat.model.reducer]
@layers = spacy.mean_max_reducer.v1
hidden_size = 128

[components.spancat.model.tok2vec]
@architectures = "spacy.Tok2VecListener.v1"
width = 96

[corpora]

[corpora.dev]
@readers = "spacy.Corpus.v1"
path = ${paths.dev}
max_length = 0
gold_preproc = false
limit = 0
augmenter = null

[corpora.tra

In [8]:
import pickle 
from pathlib import Path
import spacy
from spacy.tokens import DocBin
from spacy.util import load_config
from spacy.training.initialize import init_nlp
from spacy.training.loop import train

def convert(samples, output):
    """
    Convierte samples para el formato de entrenamiento/teste y salva en el archivo 
    output con extensión .spacy 
    """
    
    nlp = spacy.blank("es")
    
    docs = []
    for doc, annot in nlp.pipe(samples, as_tuples = True):
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label = label, alignment_mode = "contract")
            ents.append(span)
        doc.spans["sc"] = ents
        docs.append(doc)
    
    # Crea un objeto DocBin        
    doc_bin = DocBin(docs = docs)
    doc_bin.to_disk(output)

def train_model(config_path, output_path):
    """
    Entrena el modelo según config_path y lo salva en la carpeta output_path
    """
    
    config = load_config(config_path)
    model = init_nlp(config)
    train(model, output_path)
    
if __name__ == '__main__':
        
    print("CARGANDO DATOS DE ENTRENAMIENTO", end = "\n\n")
    with open("training_data_spancat.pickle", "rb") as fp:   
        training_data = pickle.load(fp)    
        print(training_data, end = "\n\n")

    print("CREANDO ARCHIVOS BINARIOS DE ENTRENAMIENTO Y TESTE", end = "\n\n")
    convert(training_data[:int(len(training_data) * 0.8)], Path("./train.spacy"))
    convert(training_data[int(len(training_data) * 0.8):], Path("./dev.spacy"))
        
    print("ENTRENANDO EL MODELO", end = "\n\n")
    train_model("config.cfg", Path("models/output-script"))
        
    print("\nCARGANDO EL MODELO", end = "\n\n")
    nlp = spacy.load('models/output-script/model-best')

    print("CARGANDO DOCUMENTOS", end = "\n\n")
    TEXT = ["Los TNOs clásicos se localizan entre 30 - 50 au.",
            "Brown especula que es muy probable que el planeta nueve sea un gigante de hielo expulsado del "
            "interior del sistema solar y de composición similar a urano y neptuno.",
            "La órbita del planeta tendría un semieje mayor de aproximadamente 700 au, unas veinte veces la "
            "distancia de neptuno al sol, aunque podría aproximarse hasta las 200 au."
            ]
    docs = nlp.pipe([i.lower() for i in TEXT], batch_size = 500)

    print("PREDICCIONES")
    for doc in docs:
        print([(span.text, span.label_) for span in doc.spans["sc"]])

CARGANDO DATOS DE ENTRENAMIENTO

[('el planeta 9 es un planeta hipotético en el sistema solar externo. ', {'entities': [(3, 12, 'Planeta')]}), ('el semieje mayor propuesto de su órbita varía entre 700 - 1500 au. ', {'entities': [(52, 65, 'Intervalo'), (58, 65, 'Magnitud')]}), ('brown ha declarado: "en realidad lo llamamos “fattie” (gordito) cuando estamos hablando entre nosotros". ', {'entities': []}), ('la capacidad para detectar el planeta nueve dependería de su ubicación y características. ', {'entities': [(30, 43, 'Planeta')]}), ('la existencia de este planeta puede inferirse por el comportamiento de un grupo de objetos transneptunianos.', {'entities': []}), ('probablemente se formó más cerca del sol, pero fue finalmente empujado más lejos por júpiter o saturno durante la época nebular, arrojándolo a los extremos exteriores del sistema solar', {'entities': [(85, 92, 'Planeta'), (95, 102, 'Planeta')]}), ('el planeta nueve es similar en tamaño y composición a los gigantes de hielo ur